In [1]:
param_market = 'US'

In [2]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./item.xlsx',
                       header=0,
                       dtype=str).fillna('')

dict_item = {}
list_job = []
for i in range(len(input_)):
    if input_.loc[i, 'Vehicle_Page'] != '':
        vehicle_page = int(input_.loc[i, 'Vehicle_Page'])
        dict_item[input_.loc[i, 'Item_Number']] = {'vehicle_page': vehicle_page,
                                                   'crawler': 0,
                                                   'dict_vehicle': {},
                                                   'vehicle_1': '',
                                                   'vehicle_2': ''}
        for j in range(vehicle_page):
            list_job.append((i, j))

print('总数量：' + str(len(list_job)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(list_job)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(666):
            try:
                i, j = list_job[a]

                data = '{"scopedContext":{"catalogDetails":{"itemId":"' + input_.loc[i, 'Item_Number'] + '","categoryId":"' + input_.loc[i, 'Category_Id'] + '","marketplaceId":"' + input_.loc[i, 'Market_Id'] + '"}}}'

                ua = UA.get_User_Agent_Requests()
                ua['Content-Type'] = 'application/json'

                if param_market == 'US':
                    url = 'https://www.ebay.com/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset=' + str(j*20) + '&module=COMPATIBILITY_TABLE&_ul=US&_stpos=91710&orig_cvip=true'
                elif param_market == 'DE':
                    url = 'https://www.ebay.de/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset=' + str(j*20) + '&module=COMPATIBILITY_TABLE&_ul=DE&_stpos=10115&orig_cvip=true'
                elif param_market == 'AU':
                    url = 'https://www.ebay.com.au/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset=' + str(j*20) + '&module=COMPATIBILITY_TABLE&ul=AU&_stpos=2019&orig_cvip=true'
                elif param_market == 'UK':
                    url = 'https://www.ebay.co.uk/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset=' + str(j*20) + '&module=COMPATIBILITY_TABLE&_ul=GB&_stpos=BS81QU&orig_cvip=true'

                resp = requests.post(url,
                                     data=data,
                                     headers=ua,
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                list_header = json_data['modules']['COMPATIBILITY_TABLE']['paginatedTable']['header']['cells']
                
                # = = = = = = = = = = = = = = =
                
                if param_market in ['US', 'AU', 'UK']:
                    name_make = 'Make'
                    name_model = 'Model'
                    name_engine = 'Engine'
                    name_year = 'Year'
                elif param_market == 'DE':
                    name_make = 'Marke'
                    name_model = 'Modell'
                    name_engine = 'Motor'
                    name_year = 'Baujahr'

                for k in range(len(list_header)):
                    if name_make in list_header[k]['textSpans'][0]['text']:
                        col_make = k
                    elif name_model in list_header[k]['textSpans'][0]['text']:
                        col_model = k
                    elif name_engine in list_header[k]['textSpans'][0]['text']:
                        col_engine = k
                    elif name_year in list_header[k]['textSpans'][0]['text']:
                        col_year = k
                        
                # = = = = = = = = = = = = = = =
                
                list_row = json_data['modules']['COMPATIBILITY_TABLE']['paginatedTable']['rows']

                # = = = = = = = = = = = = = = =
                
                for row in list_row:
                    make = row['cells'][col_make]['textSpans'][0]['text'].strip()
                    model = row['cells'][col_model]['textSpans'][0]['text'].strip()
                    engine = row['cells'][col_engine]['textSpans'][0]['text'].strip()
                    if param_market in ['US', 'AU', 'UK']:
                        list_year = [int(row['cells'][col_year]['textSpans'][0]['text'].strip())]
                    elif param_market == 'DE':
                        list_year = [int(year.split('/').strip()) for year in row['cells'][col_year]['textSpans'][0]['text'].split('-')]

                    if make not in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle']:
                        dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make] = {model: {engine: list_year}}
                    else:
                        if model not in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make]:
                            dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model] = {engine: list_year}
                        else:
                            if engine not in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model]:
                                dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine] = list_year
                            else:
                                dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine] += list_year

                    dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine] = list(set(dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine]))
                    dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine].sort()

                dict_item[input_.loc[i, 'Item_Number']]['crawler'] += 1

                # = = = = = = = = = = = = = = =
                
                if dict_item[input_.loc[i, 'Item_Number']]['vehicle_page'] == dict_item[input_.loc[i, 'Item_Number']]['crawler']:
                    list_vehicle = []
                    for make in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle']:
                        for model in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make]:
                            list_year = []
                            for engine in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model]:
                                list_year += dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine]
                            list_year = list(set(list_year))
                            list_year.sort()

                            year_min = list_year[0]
                            year_max = list_year[-1]
                            if year_min == year_max:
                                list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                            else:
                                list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))

                    list_vehicle.sort()

                    dict_item[input_.loc[i, 'Item_Number']]['vehicle_1'] = '\n'.join(list_vehicle)
                    
                    # = = = = = = = = = = = = = = =
                    
                    list_vehicle = []
                    for make in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle']:
                        for model in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make]:
                            for engine in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model]:
                                year_min = dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine][0]
                                year_max = dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine][-1]
                                if year_min == year_max:
                                    if engine == '':
                                        list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                                    else:
                                        list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + ' ' + engine)
                                else:
                                    if engine == '':
                                        list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))
                                    else:
                                        list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max) + ' ' + engine)

                    list_vehicle.sort()

                    dict_item[input_.loc[i, 'Item_Number']]['vehicle_2'] = '\n'.join(list_vehicle)
                    
                    # = = = = = = = = = = = = = = =
                    
                    df_temp = pd.DataFrame([{'Item_Number': input_.loc[i, 'Item_Number'],
                                             'Vehicle_1': dict_item[input_.loc[i, 'Item_Number']]['vehicle_1'],
                                             'Vehicle_2': dict_item[input_.loc[i, 'Item_Number']]['vehicle_2']}])

                    output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'

                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Item_Number': input_.loc[i, 'Item_Number'],
                                     'Vehicle_Page': input_.loc[i, 'Vehicle_Page'],
                                     'Category_Id': input_.loc[i, 'Category_Id'],
                                     'Market_Id': input_.loc[i, 'Market_Id']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[i, 'Item_Number'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(10):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Item_Number'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./vehicle-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Item_Number'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./vehicle_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：3214

[ok] - 232324496897
[尝试次数：1] - [剩余数量：3204] - [当前时间：16:16:29]

[ok] - 352050280888
[尝试次数：1] - [剩余数量：3203] - [当前时间：16:16:29]

[ok] - 232324493156
[尝试次数：1] - [剩余数量：3202] - [当前时间：16:16:29]

[ok] - 232320263307
[尝试次数：1] - [剩余数量：3201] - [当前时间：16:16:29]

[ok] - 232331576523
[尝试次数：1] - [剩余数量：3200] - [当前时间：16:16:29]

[ok] - 352050321424
[尝试次数：1] - [剩余数量：3199] - [当前时间：16:16:30]

[ok] - 352050321424
[尝试次数：1] - [剩余数量：3198] - [当前时间：16:16:30]

[ok] - 232321012287
[尝试次数：1] - [剩余数量：3197] - [当前时间：16:16:30]

[ok] - 232320263307
[尝试次数：1] - [剩余数量：3196] - [当前时间：16:16:30]

[ok] - 234166653667
[尝试次数：1] - [剩余数量：3195] - [当前时间：16:16:30]

[ok] - 352043697345
[尝试次数：1] - [剩余数量：3194] - [当前时间：16:16:30]

[ok] - 232331576523
[尝试次数：1] - [剩余数量：3193] - [当前时间：16:16:30]

[ok] - 232318533390
[尝试次数：1] - [剩余数量：3192] - [当前时间：16:16:31]

[ok] - 232320259365
[尝试次数：1] - [剩余数量：3191] - [当前时间：16:16:31]

[ok] - 232318533390
[尝试次数：1] - [剩余数量：3190] - [当前时间：16:16:31]

[ok] - 352043697345
[尝试次数：1] - [剩余数量：3189] - [当前时间：16:16:31]

[ok] - 352047171342
[尝试次数：1] - [剩余数量：3071] - [当前时间：16:16:59]

[ok] - 352047171342
[尝试次数：1] - [剩余数量：3070] - [当前时间：16:16:59]

[ok] - 352048143184
[尝试次数：1] - [剩余数量：3069] - [当前时间：16:17:00]

[ok] - 352048143184
[尝试次数：1] - [剩余数量：3068] - [当前时间：16:17:01]

[ok] - 352048143184
[尝试次数：1] - [剩余数量：3067] - [当前时间：16:17:01]

[ok] - 352048143184
[尝试次数：1] - [剩余数量：3066] - [当前时间：16:17:02]

[ok] - 352046196969
[尝试次数：2] - [剩余数量：3065] - [当前时间：16:17:02]

[ok] - 352048143184
[尝试次数：1] - [剩余数量：3064] - [当前时间：16:17:02]

[ok] - 234166433080
[尝试次数：1] - [剩余数量：3063] - [当前时间：16:17:02]

[ok] - 352048143184
[尝试次数：1] - [剩余数量：3062] - [当前时间：16:17:03]

[ok] - 352048143184
[尝试次数：1] - [剩余数量：3061] - [当前时间：16:17:03]

[ok] - 352048143184
[尝试次数：1] - [剩余数量：3060] - [当前时间：16:17:03]

[ok] - 352049024626
[尝试次数：1] - [剩余数量：3059] - [当前时间：16:17:03]

[ok] - 352049024626
[尝试次数：1] - [剩余数量：3058] - [当前时间：16:17:04]

[ok] - 232324450663
[尝试次数：1] - [剩余数量：3057] - [当前时间：16:17:04]

[ok] - 232324450663
[尝试次数：1] - [剩余数量：3056] - [当前时间：16:17:05]

[ok] - 2

[ok] - 234183533809
[尝试次数：1] - [剩余数量：2938] - [当前时间：16:17:41]

[ok] - 352052815520
[尝试次数：1] - [剩余数量：2937] - [当前时间：16:17:41]

[ok] - 352052815520
[尝试次数：1] - [剩余数量：2936] - [当前时间：16:17:42]

[ok] - 352052815520
[尝试次数：1] - [剩余数量：2935] - [当前时间：16:17:42]

[ok] - 352052815520
[尝试次数：1] - [剩余数量：2934] - [当前时间：16:17:42]

[ok] - 352052815520
[尝试次数：1] - [剩余数量：2933] - [当前时间：16:17:43]

[ok] - 352052815520
[尝试次数：1] - [剩余数量：2932] - [当前时间：16:17:43]

[ok] - 352052815520
[尝试次数：1] - [剩余数量：2931] - [当前时间：16:17:44]

[ok] - 234183533809
[尝试次数：1] - [剩余数量：2930] - [当前时间：16:17:44]

[ok] - 352052815520
[尝试次数：1] - [剩余数量：2929] - [当前时间：16:17:45]

[ok] - 352052815520
[尝试次数：1] - [剩余数量：2928] - [当前时间：16:17:45]

[ok] - 352052815520
[尝试次数：1] - [剩余数量：2927] - [当前时间：16:17:45]

[ok] - 352052815520
[尝试次数：1] - [剩余数量：2926] - [当前时间：16:17:45]

[ok] - 352052815520
[尝试次数：1] - [剩余数量：2925] - [当前时间：16:17:46]

[ok] - 352052815520
[尝试次数：1] - [剩余数量：2924] - [当前时间：16:17:46]

[ok] - 232362846133
[尝试次数：1] - [剩余数量：2923] - [当前时间：16:17:46]

[ok] - 3

[ok] - 232331634869
[尝试次数：1] - [剩余数量：2805] - [当前时间：16:18:23]

[ok] - 232331634869
[尝试次数：1] - [剩余数量：2804] - [当前时间：16:18:24]

[ok] - 232331634869
[尝试次数：1] - [剩余数量：2803] - [当前时间：16:18:24]

[ok] - 232331634869
[尝试次数：1] - [剩余数量：2802] - [当前时间：16:18:24]

[ok] - 232331634869
[尝试次数：1] - [剩余数量：2801] - [当前时间：16:18:24]

[ok] - 232331634869
[尝试次数：1] - [剩余数量：2800] - [当前时间：16:18:25]

[ok] - 232331634869
[尝试次数：1] - [剩余数量：2799] - [当前时间：16:18:26]

[ok] - 232331634869
[尝试次数：1] - [剩余数量：2798] - [当前时间：16:18:27]

[ok] - 352049168083
[尝试次数：2] - [剩余数量：2797] - [当前时间：16:18:27]

[ok] - 232331634869
[尝试次数：1] - [剩余数量：2796] - [当前时间：16:18:27]

[ok] - 232331634869
[尝试次数：1] - [剩余数量：2795] - [当前时间：16:18:28]

[ok] - 232331634869
[尝试次数：1] - [剩余数量：2794] - [当前时间：16:18:28]

[ok] - 232331634869
[尝试次数：1] - [剩余数量：2793] - [当前时间：16:18:28]

[ok] - 352041434811
[尝试次数：1] - [剩余数量：2792] - [当前时间：16:18:28]

[ok] - 352041434811
[尝试次数：1] - [剩余数量：2791] - [当前时间：16:18:29]

[ok] - 352041434811
[尝试次数：1] - [剩余数量：2790] - [当前时间：16:18:29]

[ok] - 3

[ok] - 232318023123
[尝试次数：1] - [剩余数量：2670] - [当前时间：16:19:06]

[ok] - 352042412302
[尝试次数：1] - [剩余数量：2669] - [当前时间：16:19:07]

[ok] - 352042412302
[尝试次数：1] - [剩余数量：2668] - [当前时间：16:19:07]

[ok] - 352042412302
[尝试次数：1] - [剩余数量：2667] - [当前时间：16:19:07]

[ok] - 232737384861
[尝试次数：2] - [剩余数量：2666] - [当前时间：16:19:08]

[ok] - 352042412302
[尝试次数：1] - [剩余数量：2665] - [当前时间：16:19:08]

[ok] - 352042412302
[尝试次数：1] - [剩余数量：2664] - [当前时间：16:19:08]

[ok] - 352042412302
[尝试次数：1] - [剩余数量：2663] - [当前时间：16:19:08]

[ok] - 352042412302
[尝试次数：1] - [剩余数量：2662] - [当前时间：16:19:08]

[ok] - 352042412302
[尝试次数：1] - [剩余数量：2661] - [当前时间：16:19:09]

[ok] - 352042412302
[尝试次数：1] - [剩余数量：2660] - [当前时间：16:19:09]

[ok] - 352042412302
[尝试次数：1] - [剩余数量：2659] - [当前时间：16:19:09]

[ok] - 352042412302
[尝试次数：1] - [剩余数量：2658] - [当前时间：16:19:10]

[ok] - 352042412302
[尝试次数：1] - [剩余数量：2657] - [当前时间：16:19:10]

[ok] - 352042412302
[尝试次数：1] - [剩余数量：2656] - [当前时间：16:19:10]

[ok] - 352041215303
[尝试次数：1] - [剩余数量：2655] - [当前时间：16:19:11]

[ok] - 3

...

[ok] - 232706689856
[尝试次数：1] - [剩余数量：663] - [当前时间：16:29:35]

[ok] - 352311986979
[尝试次数：1] - [剩余数量：662] - [当前时间：16:29:35]

[ok] - 352317737032
[尝试次数：1] - [剩余数量：661] - [当前时间：16:29:36]

[ok] - 352317737032
[尝试次数：1] - [剩余数量：660] - [当前时间：16:29:36]

[ok] - 352317737032
[尝试次数：1] - [剩余数量：659] - [当前时间：16:29:36]

[ok] - 232706689856
[尝试次数：1] - [剩余数量：658] - [当前时间：16:29:36]

[ok] - 352317737032
[尝试次数：1] - [剩余数量：657] - [当前时间：16:29:37]

[ok] - 232714941155
[尝试次数：1] - [剩余数量：656] - [当前时间：16:29:37]

[ok] - 352317737032
[尝试次数：1] - [剩余数量：655] - [当前时间：16:29:37]

[ok] - 232714941155
[尝试次数：1] - [剩余数量：654] - [当前时间：16:29:38]

[ok] - 232698814894
[尝试次数：1] - [剩余数量：653] - [当前时间：16:29:38]

[ok] - 232698814894
[尝试次数：1] - [剩余数量：652] - [当前时间：16:29:38]

[ok] - 352305901327
[尝试次数：1] - [剩余数量：651] - [当前时间：16:29:40]

[ok] - 352311985761
[尝试次数：1] - [剩余数量：650] - [当前时间：16:29:41]

[ok] - 352305981477
[尝试次数：1] - [剩余数量：649] - [当前时间：16:29:42]

[ok] - 352305981477
[尝试次数：1] - [剩余数量：648] - [当前时间：16:29:42]

[ok] - 352305981477
[尝试次

[ok] - 352312081856
[尝试次数：1] - [剩余数量：528] - [当前时间：16:30:30]

[ok] - 352312081856
[尝试次数：1] - [剩余数量：527] - [当前时间：16:30:30]

[ok] - 232706627066
[尝试次数：1] - [剩余数量：526] - [当前时间：16:30:30]

[ok] - 352312081856
[尝试次数：1] - [剩余数量：525] - [当前时间：16:30:31]

[ok] - 352312081856
[尝试次数：1] - [剩余数量：524] - [当前时间：16:30:31]

[ok] - 352312081856
[尝试次数：1] - [剩余数量：523] - [当前时间：16:30:31]

[ok] - 352312081856
[尝试次数：1] - [剩余数量：522] - [当前时间：16:30:31]

[ok] - 352312081856
[尝试次数：1] - [剩余数量：521] - [当前时间：16:30:32]

[ok] - 352312081856
[尝试次数：1] - [剩余数量：520] - [当前时间：16:30:32]

[ok] - 352312081856
[尝试次数：1] - [剩余数量：519] - [当前时间：16:30:32]

[ok] - 352312081856
[尝试次数：1] - [剩余数量：518] - [当前时间：16:30:32]

[ok] - 232706661472
[尝试次数：1] - [剩余数量：517] - [当前时间：16:30:33]

[ok] - 352311985761
[尝试次数：2] - [剩余数量：516] - [当前时间：16:30:33]

[ok] - 232706661472
[尝试次数：1] - [剩余数量：515] - [当前时间：16:30:33]

[ok] - 232706701841
[尝试次数：1] - [剩余数量：514] - [当前时间：16:30:33]

[ok] - 232706661472
[尝试次数：1] - [剩余数量：513] - [当前时间：16:30:34]

[ok] - 352317681817
[尝试次

[ok] - 352045992958
[尝试次数：1] - [剩余数量：392] - [当前时间：16:31:05]

[ok] - 352045992958
[尝试次数：1] - [剩余数量：391] - [当前时间：16:31:05]

[ok] - 352041150721
[尝试次数：1] - [剩余数量：390] - [当前时间：16:31:05]

[ok] - 352041150721
[尝试次数：1] - [剩余数量：389] - [当前时间：16:31:05]

[ok] - 352041156296
[尝试次数：1] - [剩余数量：388] - [当前时间：16:31:06]

[ok] - 352041156296
[尝试次数：1] - [剩余数量：387] - [当前时间：16:31:06]

[ok] - 352041156296
[尝试次数：1] - [剩余数量：386] - [当前时间：16:31:06]

[ok] - 232706793731
[尝试次数：2] - [剩余数量：385] - [当前时间：16:31:06]

[ok] - 352041150721
[尝试次数：1] - [剩余数量：384] - [当前时间：16:31:06]

[ok] - 352041164474
[尝试次数：1] - [剩余数量：383] - [当前时间：16:31:06]

[ok] - 232322151950
[尝试次数：1] - [剩余数量：382] - [当前时间：16:31:07]

[ok] - 232322151950
[尝试次数：1] - [剩余数量：381] - [当前时间：16:31:07]

[ok] - 232322151950
[尝试次数：1] - [剩余数量：380] - [当前时间：16:31:07]

[ok] - 352041164474
[尝试次数：1] - [剩余数量：379] - [当前时间：16:31:07]

[ok] - 232322151950
[尝试次数：1] - [剩余数量：378] - [当前时间：16:31:08]

[ok] - 232322151950
[尝试次数：1] - [剩余数量：377] - [当前时间：16:31:08]

[ok] - 232322151950
[尝试次

[ok] - 352049035826
[尝试次数：1] - [剩余数量：257] - [当前时间：16:31:43]

[ok] - 352049035826
[尝试次数：1] - [剩余数量：256] - [当前时间：16:31:43]

[ok] - 352049035826
[尝试次数：1] - [剩余数量：255] - [当前时间：16:31:44]

[ok] - 352049035826
[尝试次数：1] - [剩余数量：254] - [当前时间：16:31:44]

[ok] - 352049035826
[尝试次数：1] - [剩余数量：253] - [当前时间：16:31:44]

[ok] - 352049035826
[尝试次数：1] - [剩余数量：252] - [当前时间：16:31:45]

[ok] - 352048550689
[尝试次数：1] - [剩余数量：251] - [当前时间：16:31:45]

[ok] - 352049035826
[尝试次数：1] - [剩余数量：250] - [当前时间：16:31:45]

[ok] - 352048550689
[尝试次数：1] - [剩余数量：249] - [当前时间：16:31:45]

[ok] - 352048550689
[尝试次数：1] - [剩余数量：248] - [当前时间：16:31:46]

[ok] - 232310354369
[尝试次数：2] - [剩余数量：247] - [当前时间：16:31:46]

[ok] - 352048550689
[尝试次数：1] - [剩余数量：246] - [当前时间：16:31:46]

[ok] - 352048550689
[尝试次数：1] - [剩余数量：245] - [当前时间：16:31:46]

[ok] - 235198409070
[尝试次数：1] - [剩余数量：244] - [当前时间：16:31:46]

[ok] - 235198409070
[尝试次数：1] - [剩余数量：243] - [当前时间：16:31:47]

[ok] - 352312081856
[尝试次数：2] - [剩余数量：242] - [当前时间：16:31:47]

[ok] - 235198409070
[尝试次

[ok] - 232324954734
[尝试次数：1] - [剩余数量：122] - [当前时间：16:32:24]

[ok] - 232324954734
[尝试次数：1] - [剩余数量：121] - [当前时间：16:32:24]

[ok] - 352048185215
[尝试次数：1] - [剩余数量：120] - [当前时间：16:32:25]

[ok] - 352049086441
[尝试次数：1] - [剩余数量：119] - [当前时间：16:32:25]

[ok] - 232318035464
[尝试次数：1] - [剩余数量：118] - [当前时间：16:32:25]

[ok] - 352041389179
[尝试次数：1] - [剩余数量：117] - [当前时间：16:32:25]

[ok] - 232346555830
[尝试次数：1] - [剩余数量：116] - [当前时间：16:32:25]

[ok] - 232326034686
[尝试次数：1] - [剩余数量：115] - [当前时间：16:32:25]

[ok] - 232362846246
[尝试次数：1] - [剩余数量：114] - [当前时间：16:32:26]

[ok] - 232320907141
[尝试次数：1] - [剩余数量：113] - [当前时间：16:32:26]

[ok] - 352049086441
[尝试次数：1] - [剩余数量：112] - [当前时间：16:32:26]

[ok] - 352039651723
[尝试次数：1] - [剩余数量：111] - [当前时间：16:32:26]

[ok] - 353747644622
[尝试次数：1] - [剩余数量：110] - [当前时间：16:32:26]

[ok] - 232318526197
[尝试次数：1] - [剩余数量：109] - [当前时间：16:32:27]

[ok] - 232318029954
[尝试次数：1] - [剩余数量：108] - [当前时间：16:32:27]

[ok] - 353749325721
[尝试次数：1] - [剩余数量：107] - [当前时间：16:32:27]

[ok] - 232324981560
[尝试次